In [8]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (9.0s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_SYMBOL = 'SOLUSDC'

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

--- Logging error ---
Traceback (most recent call last):
  File "/workspaces/ethereum-prices/src/helpers.py", line 31, in fetch_object
    s3.Object(bucket_name, filename).download_fileobj(data)
  File "/workspaces/ethereum-prices/.venv/lib/python3.12/site-packages/boto3/s3/inject.py", line 890, in object_download_fileobj
    return self.meta.client.download_fileobj(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/ethereum-prices/.venv/lib/python3.12/site-packages/boto3/s3/inject.py", line 801, in download_fileobj
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/workspaces/ethereum-prices/.venv/lib/python3.12/site-packages/s3transfer/futures.py", line 103, in result
    return self._coordinator.result()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/ethereum-prices/.venv/lib/python3.12/site-packages/s3transfer/futures.py", line 266, in result
    raise self._exception
  File "/workspaces/ethereum-prices/.venv/lib/python3.12/site-packages/s3tra

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

## Option valuation model: input parameters here

In [51]:
target_period_hours = 2 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: Sat 17 Feb, 08:00 (47 hours left)
current price: 2781.08
-------------------------------
trading put 2750 and call 2800 (hit ratio: 46%)
cost: 0.024 / value: 0.034, benefit/cost = 1.5x
($) cost: 65.36 / value: 95.86, average gain = 30.50
-------------------------------
trading put 2725 and call 2825 (hit ratio: 43%)
cost: 0.017 / value: 0.028, benefit/cost = 1.6x
($) cost: 47.28 / value: 77.97, average gain = 30.69
-------------------------------
trading put 2700 and call 2850 (hit ratio: 39%)
cost: 0.012 / value: 0.023, benefit/cost = 1.8x
($) cost: 34.49 / value: 63.48, average gain = 28.99
-------------------------------
trading put 2675 and call 2875 (hit ratio: 34%)
cost: 0.009 / value: 0.019, benefit/cost = 2.0x
($) cost: 25.31 / value: 51.71, average gain = 26.40


[{'put': 2725.0, 'call': 2825.0, 'quantity': 0},
 {'put': 2700.0, 'call': 2850.0, 'quantity': 0}]

In [52]:
simulation.options_chain

,value_call,value_call_pct,call_bid,call_ask,value_put,value_put_pct,put_bid,put_ask
strike,,,,,,,,
2675.0,137.998264,NaN,0.0350,0.0510,21.152531,0.007606,0.0030,0.0031
2700.0,118.066824,NaN,0.0290,0.0380,26.221092,0.009428,0.0044,0.0049
2725.0,99.400539,NaN,0.0255,0.0300,32.554806,0.011706,0.0065,0.0070
2750.0,82.386106,NaN,0.0205,0.0235,40.540373,0.014577,0.0095,0.0105
2775.0,67.517819,NaN,0.0160,0.0170,50.672087,0.018220,0.0130,0.0140
2800.0,55.317358,0.019891,0.0120,0.0130,63.471626,NaN,0.0180,0.0190
2825.0,45.415739,0.016330,0.0095,0.0100,78.570006,NaN,0.0050,NaN
2850.0,37.258202,0.013397,0.0065,0.0075,95.412470,NaN,0.0085,NaN
2875.0,30.555679,0.010987,0.0050,0.0060,113.709946,NaN,0.0175,NaN


In [53]:
#print(simulation.evaluate_put(2300), simulation.get_put_bid(2300))
put_strike, call_strike = 2700, 2850
print(f" put {put_strike}: value {simulation.evaluate_put(put_strike):.02f} / cost {simulation.get_put_bid(put_strike):.02f}")
print(f"call {call_strike}: value {simulation.evaluate_call(call_strike):.02f} / cost {simulation.get_call_bid(call_strike):.02f}")

 put 2700: value 26.22 / cost 12.24
call 2850: value 37.26 / cost 18.08


# Sanity checks

In [41]:
38/31.5

1.2063492063492063